In [163]:
import matplotlib.pyplot as plt
import numpy as np
import random
import csv

In [164]:
items = []

with open('data/main_data.csv') as csvfile:    
	csvReader = csv.reader(csvfile)    
	for row in csvReader:        
		items.append(row[1])

temp = items[673:]

data = list(map(int, temp))

print(data)

[1745, 2116, 2604, 3205, 2925, 2927, 4878, 7010, 8077, 9905, 11588, 12422, 10185, 16021, 17895, 27197, 32211, 33729, 36057, 26121, 37492, 46843, 40618, 40489, 55209, 44526, 36501, 57049, 64718, 63956, 59635, 59384, 48484, 34418, 57491, 61488, 57426, 49447, 46643, 34976, 25054, 24728, 40920, 37259, 26347, 30156, 24867, 21380, 30148, 26336]


In [165]:
def generasi_awal(pop_size, gene_per_pop):
    population_size = (pop_size, gene_per_pop)
    gene = np.random.uniform(low=0.1, high=0.9, size=population_size)
    return gene

In [166]:
def day_predict(pop_size, gene_per_pop):
    day = []
    for i in range(2,pop_size):
        row = []
        for j in range(0, gene_per_pop):
            if j == 0:
                row.append(data[i-1])
            elif j == 1:
                row.append(data[i-2])
            elif j == 2:
                row.append(data[i])
        day.append(row)
    return day


In [167]:
def y_pred(pop_size, gene, day_predict, gene_per_pop):
    y_pred_indv = []
    for i in range (0,pop_size-2):
        row = []
        for j in gene:
            total = 0
            z = 0
            for k in j:
                if k<gene_per_pop-1:
                    g = k*day_predict[i][z]
                else:
                    g=k
                z=z+1
                total = total+g
            row.append(total)
        y_pred_indv.append(row)
    return y_pred_indv

In [168]:
def fitness_func(ps, day_predict, hasil_prediksi, gene_per_pop, dp):
    mape = []
    fitn = []
    data_y = dp[:, (gene_per_pop-1)]
    for i in range(0, ps):
        row = []
        for j in range(0,len(data_y)):
            if hasil_prediksi[j][i]>0:
                row.append(abs(hasil_prediksi[j][i]-data_y[j]/hasil_prediksi[j][i]))
            else:
                row.append(abs(0))
        mape.append(row)
    for i in range(0, len(mape)):
        f = 0
        for j in range(0,ps-2):
            f = f + mape[i][j]
        f=f/len(mape)
        fitn.append(1/(f+0.01))
    return fitn

In [169]:
def probability(fitness):
    prob = []
    sumF = 0
    for i in range(0,len(fitness)):
        sumF = sumF + fitness[i]
    for i in range(0,len(fitness)):
        prob.append(fitness[i]/sumF*100)
    return prob

In [170]:
def rangePc(probs):
    ranges = []
    x = 0
    y = 0
    minArray = []
    maxArray = []
    for i in range(0, len(probs)):
        if i == 0:
            x = 0
        else:
            x = x + probs[i-1]
        y = y + probs[i]
        minArray.append(x)
        maxArray.append(y)
    ranges = np.vstack((minArray, maxArray)).T
    return ranges

In [171]:
def round_wheel(pc, fitness, ranges):
    rW = []
    for i in range(0, int(pc * 50)):
        rNum = random.uniform(0, 100)
        for j in range(0, len(ranges)):
            if rNum >= ranges[j][0] and rNum <= ranges[j][1]:
                rW.append(j)
    return rW

In [172]:
def cross_over(pc, rw, gA):
    co = []
    x = 0
    y = 0
    for i in range(0, len(rw)):
        row = []
        for j in range(0, gene_per_pop):
            row.append(gA[rw[i]][j])
        co.append(row)
    for i in range(0, len(rw),2):
        if i % 2 == 0 and i == len(rw)-1:
            co[i][1], co[0][1] = (co[0][1] * (1-pc)) + (co[i][1] * pc), (co[i][1] * (1-pc)) + (co[0][1] * pc)
        else:
            co[i][1], co[i + 1][1] = (co[i + 1][1] * (1-pc)) + (co[i][1] * pc), (co[i][1] * (1-pc)) + (co[i + 1][1] * pc)
    return co

In [173]:
def evalGen(generasiAwal, gene_per_pop, fitness, offSpring, fitnessOffspring):
    evalIndv = []
    indvAwal = []
    indvOffspring = []
    for i in range(0, len(generasiAwal)):
        row = []
        for j in range(0, gene_per_pop+1):
            if j == gene_per_pop:
                row.append(fitness[i])
            else:
                row.append(generasiAwal[i][j])
        indvAwal.append(row)
    for i in range(0, len(offSpring)):
        row = []
        for j in range(0, gene_per_pop+1):
            if j == gene_per_pop:
                row.append(fitnessOffspring[i])
            else:
                row.append(offSpring[i][j])
        indvOffspring.append(row)
    evalIndv = indvAwal + indvOffspring
    return sorted(evalIndv, key=lambda x: x[3], reverse=True)

In [174]:
def eval_elitism(evalIndividu, pop_size):
    for i in range(0, len(evalIndividu)-pop_size):
        evalIndividu.pop()
    return evalIndividu

In [175]:
pop_size = 50
gene_per_pop = 3
pc = 0.7
generasiAwal = generasi_awal(pop_size, gene_per_pop)
generasiAwal

array([[0.16115568, 0.27470012, 0.37998551],
       [0.72986622, 0.88011618, 0.70945474],
       [0.65353706, 0.3824792 , 0.11772097],
       [0.37323466, 0.43280828, 0.5004497 ],
       [0.39553824, 0.89085373, 0.2671115 ],
       [0.55482904, 0.51536015, 0.28601357],
       [0.41809873, 0.31516964, 0.59673269],
       [0.64858523, 0.49532712, 0.53139426],
       [0.61116982, 0.22903363, 0.39956164],
       [0.76754616, 0.6789716 , 0.35180166],
       [0.89565279, 0.54383551, 0.52011027],
       [0.2620072 , 0.67419622, 0.6579559 ],
       [0.48933874, 0.73561756, 0.23612987],
       [0.6280277 , 0.40005588, 0.67704292],
       [0.89395203, 0.48180855, 0.55597115],
       [0.11784255, 0.14477937, 0.10135118],
       [0.72052702, 0.12693554, 0.72508523],
       [0.13538408, 0.6346972 , 0.69003381],
       [0.41339172, 0.26757085, 0.25564324],
       [0.24841645, 0.24534362, 0.25292172],
       [0.63812878, 0.38587873, 0.81543188],
       [0.73559163, 0.45723367, 0.48672949],
       [0.

In [176]:
dayPredict = day_predict(pop_size, gene_per_pop)
dayPredict = [list( map(int, i) ) for i in dayPredict]
print (dayPredict)

[[2116, 1745, 2604], [2604, 2116, 3205], [3205, 2604, 2925], [2925, 3205, 2927], [2927, 2925, 4878], [4878, 2927, 7010], [7010, 4878, 8077], [8077, 7010, 9905], [9905, 8077, 11588], [11588, 9905, 12422], [12422, 11588, 10185], [10185, 12422, 16021], [16021, 10185, 17895], [17895, 16021, 27197], [27197, 17895, 32211], [32211, 27197, 33729], [33729, 32211, 36057], [36057, 33729, 26121], [26121, 36057, 37492], [37492, 26121, 46843], [46843, 37492, 40618], [40618, 46843, 40489], [40489, 40618, 55209], [55209, 40489, 44526], [44526, 55209, 36501], [36501, 44526, 57049], [57049, 36501, 64718], [64718, 57049, 63956], [63956, 64718, 59635], [59635, 63956, 59384], [59384, 59635, 48484], [48484, 59384, 34418], [34418, 48484, 57491], [57491, 34418, 61488], [61488, 57491, 57426], [57426, 61488, 49447], [49447, 57426, 46643], [46643, 49447, 34976], [34976, 46643, 25054], [25054, 34976, 24728], [24728, 25054, 40920], [40920, 24728, 37259], [37259, 40920, 26347], [26347, 37259, 30156], [30156, 26347,

In [177]:
yPred = y_pred(pop_size, generasiAwal, dayPredict, gene_per_pop)
yPred

[[1809.8394087088323,
  4927.61980944244,
  2356.8560381096927,
  2848.18601760569,
  3087.057018856004,
  2818.101045330725,
  2988.5598617442784,
  3620.502840587369,
  2733.357540216927,
  3725.024647723001,
  4198.561424877112,
  3444.1967897652644,
  2933.9756007246147,
  3790.023889956804,
  4180.107309778659,
  765.9133120448521,
  3634.2596168812706,
  3190.8673510411563,
  2007.3430056450902,
  1612.382004823733,
  4147.023500425572,
  3621.8282313703303,
  1597.1704489320127,
  3427.820652177111,
  3322.184876704183,
  4052.0202263390433,
  1682.8998083405331,
  3524.96053948576,
  2287.611310623227,
  2102.62594914887,
  2690.0705470370845,
  3565.958561324441,
  4635.384098346447,
  2771.7705286582404,
  3744.006055489779,
  3030.4954300706504,
  4422.26744741117,
  4172.031919183812,
  4525.734832309715,
  2826.2615706579113,
  4144.664407066262,
  2936.6471226251297,
  3015.901813800096,
  2180.9074378378455,
  2494.3275537697955,
  4441.374924596256,
  3276.201372406901,

In [178]:
dayPredict = np.array(dayPredict)
fitness = fitness_func(pop_size, day_predict, yPred, gene_per_pop, dayPredict)
fitness

[4.015181848931022e-05,
 1.416965536136424e-05,
 2.855388942927146e-05,
 2.510252282428698e-05,
 2.127841858995022e-05,
 2.4270805917871697e-05,
 2.458996327700161e-05,
 1.958362326684154e-05,
 2.6407063153909605e-05,
 1.8307305543843372e-05,
 1.6750564978124777e-05,
 2.059808807608452e-05,
 2.2598415040179826e-05,
 1.919492436299242e-05,
 1.6977267396480453e-05,
 9.03895924370639e-05,
 2.073288132863004e-05,
 2.2472665417036763e-05,
 3.504826202170786e-05,
 4.395317533120836e-05,
 1.7772076483661254e-05,
 1.9534064130393737e-05,
 4.302182751920694e-05,
 2.0867004148550944e-05,
 2.2201859595465334e-05,
 1.7554113335118995e-05,
 4.0379592040649504e-05,
 2.068237914901915e-05,
 3.238915778725939e-05,
 3.418865676239622e-05,
 2.3900629113681202e-05,
 1.95494303027067e-05,
 1.4956213078108638e-05,
 2.6126252185311106e-05,
 1.9769966015221347e-05,
 2.6007915975913608e-05,
 1.653522597305456e-05,
 1.652870229174608e-05,
 1.568418822903214e-05,
 2.57167073528381e-05,
 1.702221367945942e-05,
 

In [179]:
yProb = probability(fitness)
yProb

[3.212600402001714,
 1.1337329720760683,
 2.2846346718670345,
 2.008486239247964,
 1.7025145731378102,
 1.941939463325157,
 1.9674756681303152,
 1.5669117451418844,
 2.112864246149168,
 1.4647918665347057,
 1.340234983299855,
 1.6480804238371782,
 1.8081292448064785,
 1.535811428848719,
 1.358373745322255,
 7.232191515245895,
 1.6588654112576147,
 1.7980678502031802,
 2.804258060948771,
 3.516752018987262,
 1.421967477538181,
 1.5629464527176458,
 3.4422336417925714,
 1.6695967564720342,
 1.7764003162289366,
 1.4045279561173805,
 3.2308248667988133,
 1.6548246646955853,
 2.591499594359569,
 2.7354799008173045,
 1.9123211248613408,
 1.5641759206024688,
 1.196666501168489,
 2.0903961870537566,
 1.5818212762883865,
 2.0809279495445896,
 1.3230054230885433,
 1.3224834546701083,
 1.25491276125075,
 2.0576279602848797,
 1.3619699572050408,
 2.059444679626921,
 1.7416833635263775,
 2.2935223388545745,
 2.270646577617306,
 1.3360204015850827,
 1.7835720351455826,
 1.3383180426519055,
 1.775346

In [180]:
yRanges = rangePc(yProb)
yRanges

array([[  0.        ,   3.2126004 ],
       [  3.2126004 ,   4.34633337],
       [  4.34633337,   6.63096805],
       [  6.63096805,   8.63945429],
       [  8.63945429,  10.34196886],
       [ 10.34196886,  12.28390832],
       [ 12.28390832,  14.25138399],
       [ 14.25138399,  15.81829573],
       [ 15.81829573,  17.93115998],
       [ 17.93115998,  19.39595185],
       [ 19.39595185,  20.73618683],
       [ 20.73618683,  22.38426725],
       [ 22.38426725,  24.1923965 ],
       [ 24.1923965 ,  25.72820793],
       [ 25.72820793,  27.08658167],
       [ 27.08658167,  34.31877319],
       [ 34.31877319,  35.9776386 ],
       [ 35.9776386 ,  37.77570645],
       [ 37.77570645,  40.57996451],
       [ 40.57996451,  44.09671653],
       [ 44.09671653,  45.51868401],
       [ 45.51868401,  47.08163046],
       [ 47.08163046,  50.5238641 ],
       [ 50.5238641 ,  52.19346086],
       [ 52.19346086,  53.96986118],
       [ 53.96986118,  55.37438913],
       [ 55.37438913,  58.605214  ],
 

In [181]:
roundWheel = round_wheel(pc, fitness, yRanges)
roundWheel

[25,
 8,
 33,
 15,
 8,
 24,
 30,
 22,
 15,
 4,
 23,
 35,
 2,
 25,
 41,
 15,
 37,
 43,
 27,
 19,
 38,
 15,
 15,
 19,
 28,
 41,
 3,
 16,
 34,
 31,
 45,
 28,
 9,
 32,
 7]

In [182]:
offSpring = cross_over(pc, roundWheel, generasiAwal)
offSpring

[[0.715828022017666, 0.4614361320390634, 0.6123372645926882],
 [0.6111698228899625, 0.32240982920858696, 0.39956163755943763],
 [0.25573529063330913, 0.34436796206459036, 0.5685287281354964],
 [0.11784255053737668, 0.2303173398597263, 0.1013511785188773],
 [0.6111698228899625, 0.2709835797715834, 0.39956163755943763],
 [0.40173904139788386, 0.32691684122290876, 0.7021630265928812],
 [0.43975115050795155, 0.6845791144152101, 0.12759633046109436],
 [0.15435342772326477, 0.5067715015120374, 0.23769125926194568],
 [0.11784255053737668, 0.3686016812146458, 0.1013511785188773],
 [0.3955382420979795, 0.6670314252260692, 0.2671114954232865],
 [0.267069267880911, 0.4704509620534938, 0.6810171945350715],
 [0.2584304323175278, 0.26537254525240117, 0.8790237794968324],
 [0.6535370591174494, 0.4298217270768001, 0.11772097289230495],
 [0.715828022017666, 0.49294509375753215, 0.6123372645926882],
 [0.5622941038613677, 0.11394293485252559, 0.6033270461673566],
 [0.11784255053737668, 0.1315637567688413

In [183]:
yPredOffspring = y_pred(pop_size, offSpring, dayPredict, gene_per_pop)
len(yPredOffspring)

48

In [184]:
fitnessOffspring = fitness_func(len(yPredOffspring[0]), day_predict, yPredOffspring, gene_per_pop, dayPredict)
fitnessOffspring

[2.0226797379453887e-05,
 2.7204021465335507e-05,
 3.075910246836791e-05,
 8.179211866050924e-05,
 2.8246992618956286e-05,
 2.5060777664412386e-05,
 2.9610720889879535e-05,
 4.0878953624624956e-05,
 6.298718098297418e-05,
 2.7675207813117194e-05,
 2.53807538892541e-05,
 2.533225728143589e-05,
 3.067079327935259e-05,
 1.9892200476568135e-05,
 2.7923420447839315e-05,
 0.00010395868299612484,
 1.7906629193598445e-05,
 3.375244854014642e-05,
 2.4101931241268545e-05,
 4.334687467074962e-05,
 1.8585266583027384e-05,
 7.239801098761867e-05,
 9.289777323875653e-05,
 5.0563850622570574e-05,
 3.6064121835431425e-05,
 2.791071767458458e-05,
 2.970373992948027e-05,
 2.15421379949905e-05,
 1.9832546860922952e-05,
 2.3875776471279158e-05,
 1.8794042058106706e-05,
 3.401314898279851e-05,
 1.9846895102758596e-05,
 1.6969455008071527e-05,
 2.1846889268875423e-05]

In [185]:
evalIndividu = evalGen(generasiAwal, gene_per_pop, fitness, offSpring, fitnessOffspring)
evalIndividu

[[0.11784255053737668,
  0.13156375676884138,
  0.1013511785188773,
  0.00010395868299612484],
 [0.11784255053737668,
  0.17494864824591322,
  0.1013511785188773,
  9.289777323875653e-05],
 [0.11784255053737668,
  0.14477937320607825,
  0.1013511785188773,
  9.03895924370639e-05],
 [0.11784255053737668,
  0.2303173398597263,
  0.1013511785188773,
  8.179211866050924e-05],
 [0.11784255053737668,
  0.2904171819961429,
  0.1013511785188773,
  7.239801098761867e-05],
 [0.11784255053737668,
  0.3686016812146458,
  0.1013511785188773,
  6.298718098297418e-05],
 [0.24841645416442706,
  0.21517434829902649,
  0.25292172238306154,
  5.0563850622570574e-05],
 [0.24841645416442706,
  0.24534362333886142,
  0.25292172238306154,
  4.395317533120836e-05],
 [0.24841645416442706,
  0.3399295668710469,
  0.25292172238306154,
  4.334687467074962e-05],
 [0.15435342772326477,
  0.3734157918346578,
  0.23769125926194568,
  4.302182751920694e-05],
 [0.15435342772326477,
  0.5067715015120374,
  0.23769125926

In [186]:
evalElitism = eval_elitism(evalIndividu, pop_size)
evalElitism

[[0.11784255053737668,
  0.13156375676884138,
  0.1013511785188773,
  0.00010395868299612484],
 [0.11784255053737668,
  0.17494864824591322,
  0.1013511785188773,
  9.289777323875653e-05],
 [0.11784255053737668,
  0.14477937320607825,
  0.1013511785188773,
  9.03895924370639e-05],
 [0.11784255053737668,
  0.2303173398597263,
  0.1013511785188773,
  8.179211866050924e-05],
 [0.11784255053737668,
  0.2904171819961429,
  0.1013511785188773,
  7.239801098761867e-05],
 [0.11784255053737668,
  0.3686016812146458,
  0.1013511785188773,
  6.298718098297418e-05],
 [0.24841645416442706,
  0.21517434829902649,
  0.25292172238306154,
  5.0563850622570574e-05],
 [0.24841645416442706,
  0.24534362333886142,
  0.25292172238306154,
  4.395317533120836e-05],
 [0.24841645416442706,
  0.3399295668710469,
  0.25292172238306154,
  4.334687467074962e-05],
 [0.15435342772326477,
  0.3734157918346578,
  0.23769125926194568,
  4.302182751920694e-05],
 [0.15435342772326477,
  0.5067715015120374,
  0.23769125926

In [188]:
evalElitism[0][0]
d51 = round((evalElitism[0][0] * data[49]) + (evalElitism[0][1] * data[48]) + evalElitism[0][2])
d52 = round((evalElitism[1][0] * d51) + (evalElitism[1][1] * data[49]) + evalElitism[1][2])
print(f'prediksi covid pada hari ke-51 adalah sebanyak {d51}')
print(f'\nprediksi covid pada hari ke-52 adalah sebanyak {d52}')

prediksi covid pada hari ke-51 adalah sebanyak 7070

prediksi covid pada hari ke-52 adalah sebanyak 5441
